In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
import time

In [2]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [3]:
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
print(is_cuda_gpu_available)
print(is_cuda_gpu_min_3)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True


In [4]:
df = pd.read_csv('data/price_informations.csv')

In [5]:
df

,Date,Close,Open,High,Low,Vol,Change %
0,7-Nov-10,0.3,0.4,0.5,0.1,341460.0,-0.29
1,14-Nov-10,0.3,0.3,0.3,0.2,146410.0,0.00
2,21-Nov-10,0.3,0.3,0.3,0.3,73380.0,0.00
3,28-Nov-10,0.2,0.3,0.3,0.2,136930.0,-0.28
4,5-Dec-10,0.2,0.2,0.2,0.2,66890.0,0.00
...,...,...,...,...,...,...,...
160,1-Dec-13,697.0,1205.7,1239.9,576.0,362230.0,-0.42
161,8-Dec-13,908.9,697.0,1067.7,653.0,144170.0,0.30
162,15-Dec-13,640.5,908.9,927.9,454.9,293970.0,-0.30
163,22-Dec-13,762.0,640.5,836.8,615.0,93740.0,0.19


In [6]:
dates = df["Date"]
df.drop("Date",axis= 1, inplace = True)
Y = df["Close"]
X = df.drop(["Close"],axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

In [7]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range (60,X_scaled.shape[0]):
    if i < X_scaled.shape[0]*0.8:
        x_train.append(X_scaled[i-60:i])
        y_train.append(Y[i])
    else:
        x_test.append(X_scaled[i-60:i])
        y_test.append(Y[i])

x_train , y_train, x_test, y_test =np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

In [8]:
epoch_count = 1000
def model_creator(input_shape):
    model = Sequential()  
    model.add(Flatten())
    model.add(LSTM(units = 64, activation = 'relu', return_sequences = True))
    model.add(Dropout(0.15))
    
    model.add(LSTM(units = 32, activation = 'relu', return_sequences = True))
    model.add(Dropout(0.15))
    
    model.add(LSTM(units = 16, activation = 'relu', return_sequences = True))
    model.add(Dropout(0.15))
    
    model.add(LSTM(units = 8, activation = 'relu'))
    model.add(Dropout(0.15))
    
    model.add(Dense(units= 1))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics = ['mape'])
    return model

In [9]:
x_train.shape

(72, 60, 5)

In [10]:
y_train.shape

(72,)

In [11]:
#x_train =np.reshape(x_train,(x_train.shape[1]*x_train.shape[2] , x_train.shape[1] ))

ValueError: cannot reshape array of size 21600 into shape (300,60)

In [14]:
model = model_creator((x_train.shape[0] , 1))
model.fit(x_train,y_train, epochs=epoch_count, batch_size=1, verbose=1)

Epoch 1/1000


ValueError: in user code:

    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\anaconda3\envs\senior\lib\site-packages\keras\engine\input_spec.py", line 214, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

In [ ]:
model.summary()